In [2]:
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt


### IMPORT YOLO Packages
### IMPORT WANDB AND MANAGE MODEL TRAINING MONITORING ## ALSO AUTOML

In [3]:
### LOGGER ###
import sys
import logging

logging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"
log_dir = "logs"
log_filepath = os.path.join(log_dir,"running_logs.log")
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    level= logging.INFO,
    format= logging_str,

    handlers=[
        logging.FileHandler(log_filepath),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger("license_plate_classification")

In [4]:
#PLAYING WITH ENV VARIABLE SO THAT WE CAN EASILY IMPORT PACKAGES OR FILES## 
from pathlib import Path
ENV_FILE = Path.cwd().parent / ".env"
PROJECT_PATH = Path.cwd().parent
CONFIGS_PATH = Path.cwd().parent / "src" /"configs"

CONFIG = CONFIGS_PATH / "config.yaml"
CONFIG.exists()

True

In [5]:
### LOAD YAML PARAMETERS
import yaml


with open(CONFIG) as file:
    data = yaml.safe_load(file)
    logger.info(f"yaml file: {CONFIG} loaded successfully")

folder_path= "data/License_coco"
data_path = PROJECT_PATH / folder_path


[2024-10-26 01:25:27,200: INFO: 667455139: yaml file: /home/vikram/Documents/portfolio/license_plate_classification/src/configs/config.yaml loaded successfully]


In [6]:
for path in data_path.iterdir():
    print(path)

/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco/README.roboflow.txt
/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco/train
/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco/test
/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco/valid
/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco/README.dataset.txt


In [6]:
## What kind of a data loader for Yolo which requires to create folders?
## Maybe create a class that will create the folders, add augmentations to training images 
# and return back the path of folders.
# How to modify bounding boxes training data for augmentations?
# Also need to think of bounding boxes 



In [12]:
raw_data_folder = Path(PROJECT_PATH/'data'/'License_coco')
raw_data_folder

PosixPath('/home/vikram/Documents/portfolio/license_plate_classification/data/License_coco')

In [8]:
## NOT REQUIRED
'''
import albumentations as A
import cv2

transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco'))

'''

"\nimport albumentations as A\nimport cv2\n\ntransform = A.Compose([\n    A.RandomCrop(width=450, height=450),\n    A.HorizontalFlip(p=0.5),\n    A.RandomBrightnessContrast(p=0.2),\n], bbox_params=A.BboxParams(format='coco'))\n\n"

In [9]:
## ADD TO UTILS and add typing
import json
from collections import defaultdict
import numpy as np

from typing import List


def coco_image_annotations(anno_file):
    if not Path.exists(anno_file):
        raise FileNotFoundError(anno_file)
    
    ## Load json and convert to file in yolo format

    with open(anno_file, 'r') as f:
        data = json.load(f)
    
    images = {"{:g}".format(x["id"]): x for x in data["images"]}
    img_to_anno=defaultdict(list)
    for anno in data['annotations']:
        img_to_anno[f"{anno['image_id']}"].append(anno)

    return images, img_to_anno 


def coco_to_yolo_bbox(bbox:List, img_width, img_height)->List:
    """
    Return bounding boxes in yolo format given List of bbox in coco format
    """
   
    # The COCO box format is [top left x, top left y, width, height]
    box = np.array(bbox, dtype=np.float64)
    if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
        return None
    
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= img_width # normalize x
    box[[1, 3]] /= img_height  # normalize y

    return box.tolist()

## ADD 
def augment_image(img, bbox:List, augment_fn=None, bounding_box_type=None):
    """
    Return a tuple of image and bounding boxes in specified formataugment_image_bounding_box
    """
    pass

In [10]:
test_list = [98,345,322,117]
w = 640
h = 480
coco_to_yolo_bbox(test_list, w, h)

[0.4046875, 0.840625, 0.503125, 0.24375]

In [13]:
train_folder = raw_data_folder/'test'
labels_folder = raw_data_folder/'test/labels'
train_anno_file = train_folder/'_annotations.coco.json'

images, img_to_anno = coco_image_annotations(train_anno_file)

In [14]:
images['200'], img_to_anno['200'][0]['bbox']

({'id': 200,
  'license': 1,
  'file_name': 'CarLongPlateGen3793_jpg.rf.318f593c294007d7ceadcd771018e075.jpg',
  'height': 303,
  'width': 472,
  'date_captured': '2023-01-30T07:40:17+00:00'},
 [251, 219, 79, 49])

In [ ]:
from tqdm import tqdm
for img_id, anns in tqdm(img_to_anno.items()):
    img = images[img_id]
    h, w, f = img["height"], img["width"], img["file_name"]

    bboxes = []
    bboxes_augmented = []
    # AUGMENT THE IMAGE and get augmented bounded boxes
    
    for ann in anns: 
        bboxes.append(coco_to_yolo_bbox(ann['bbox'],w,h))

    
    
    ## Can look for a better and easy way and make it a function of writing yolo annotations
    with open((labels_folder / f).with_suffix(".txt"), "a") as file:
        for i in range(len(bboxes)):
            line = (0,*(bboxes[i]),)  # box           
            file.write(("%g " * len(line)).rstrip() % line + "\n")

In [13]:
def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), "License Plate")
    
    plt.imshow(np.array(image))
    plt.show()

In [13]:
## CONFIG VARIABLES



In [15]:
from IPython.display import display, Image

In [16]:
import ultralytics
ultralytics.checks()



Ultralytics 8.3.23 🚀 Python-3.10.15 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3799MiB)
Setup complete ✅ (12 CPUs, 30.7 GB RAM, 309.9/1823.8 GB disk)


In [ ]:
### Training the model and running validation files
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
results = model.train(data=CONFIG, epochs=150, imgsz=420, plots=True, name='size_420_AdamW', save_period=10, optimizer='AdamW')


In [17]:
import torch 
import torchvision
print(torch.__version__) 
print(torchvision.__version__)

2.5.0
0.20.0


In [ ]:
from ultralytics import YOLO
model = YOLO(PROJECT_PATH/"notebooks/runs/detect/run2/weights/best.pt")
results = model.tune(data=CONFIG, epochs=30, imgsz=640,plots=True, iterations=300, optimizer="AdamW" )

In [18]:
## Inference
from ultralytics import YOLO

inf_model_path=PROJECT_PATH/"notebooks/runs/detect/size_420/weights/best.pt"
inf_model = YOLO(inf_model_path)
inp_source = raw_data_folder/'test/images'
#source2 = raw_data_folder/'test/images/00a7d31c6cc6b7f3_jpg.rf.641695200cda83be76f64c5402215f27.jpg'

def inference(source, model="yolo11n.pt", save_path=None, show=None ):
    """
    Source can be directory, image file, np.array containing image
    """
    if source: ## Depending on type , Have to change inference
        out = inf_model(inp_source, stream=True) ## FOR DIRECTORY
    

    # Run inference on the source
    files = Path(inp_source).glob('*')
    for file in files:
        out2 = inf_model(file)
        break



In [51]:
from PIL import Image
# Visualize the results
for i, r in enumerate(out2):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
    # Show results to screen (in supported environments)
    r.show()
    # Save results to disk
    r.save(filename=f"out2.jpg")


In [ ]:
## Abstraction of the model into a class

In [ ]:
## PIPELINING THE CODE
## ADD APP for inference
